Installing necessary packages and importing the libraries

In [ ]:
!pip install pytesseract
!apt-get install poppler-utils
!pip install pdf2image
!pip install tesseract
!pip install Pillow==9.0.0
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (221 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-an

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,837 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121779 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

Import

In [ ]:
# Import libraries
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
from pytesseract import image_to_string
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
import nltk

Connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 converting the scanned document to image

In [ ]:
# Path of the pdf

PDF_file = "/content/drive/MyDrive/ML/greenit.pdf"

'''
Module #1: Converting PDF to images
'''

# Store all the pages of the PDF in a variable
pages = convert_from_path("/content/drive/MyDrive/ML/greenit.pdf", 500)

#Counter to store images of each page of PDF to image
image_counter = 1

# Iterate through all the pages stored above

for page in pages:
  filename = "page_" + str(image_counter) + ".jpg"
  page.save(filename, 'JPEG')
  image_counter = image_counter + 1

Performing OCR on the converted image

In [ ]:
#Variable to get count of total number of pages
filelimit = image_counter - 1

#Creating a text file to write the output
outfile = "green.txt"

#Open the file in append mode so that
#All contents of all images are added to the same file
f = open(outfile, "a")

# Iterate from 1 to total number of pages
for i in range(1, filelimit + 1):
   filename = "page_" + str(i) + ".jpg"
   text = str(((pytesseract.image_to_string(Image.open(filename)))))
   text = text.replace('-\n', '')
   f.write(text)

#Close the file after writing all the text.
f.close()

Using the TextRank Algorithm to perform text summarization

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
sentences=sent_tokenize(text)

In [ ]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens = [[word for word in sentence.split(' ') if word not in stop_words] for sentence in sentences_clean]

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

!unzip glove*.zip

--2024-02-24 11:43:50--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-02-24 11:43:50--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-02-24 11:43:50--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)

400000

In [ ]:
#remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase

clean_sentences = [s.lower() for s in clean_sentences]

<ipython-input-13-f94aa3d87f1d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  word_embeddings[word] = coefs
f.close()

sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split()) +0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

After extracting the word embeddings we form a similarity matrix by calculating the cosine similarity between each pair of sentences. After this is done, the matrix is then converted into graph with the nodes representing the sentences and the similarity representing the edges.

In [ ]:
#similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i !=j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
print(scores)

{0: 0.0340658245685241, 1: 0.03555490550574928, 2: 0.03580357682202379, 3: 0.03404305498220462, 4: 0.0341671269019479, 5: 0.029286455576598282, 6: 0.034327750657352975, 7: 0.0341523657563127, 8: 0.03442386293176325, 9: 0.035862146258779995, 10: 0.03552233809744091, 11: 0.033101527273409016, 12: 0.0350821712654312, 13: 0.03638742831170936, 14: 0.033841042959568524, 15: 0.021807373621909404, 16: 0.03170074800961943, 17: 0.029190336221397467, 18: 0.029190794978414168, 19: 0.0347391897372993, 20: 0.032207321666540424, 21: 0.035205561724599606, 22: 0.02602627074313604, 23: 0.03719969706244466, 24: 0.017980833380062466, 25: 0.035157115727390026, 26: 0.004769533835018383, 27: 0.004769533835018383, 28: 0.004769533835018383, 29: 0.033588700518631116, 30: 0.025242654482203624, 31: 0.026274044494300156, 32: 0.03095314540088547, 33: 0.023606032857295445}


In [ ]:
top_sentence = {sentence:scores[index] for index, sentence in enumerate(sentences)}
top = dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])

Printing the Summary



In [ ]:
summary = ""
for sent in sentences:
  if sent in top.keys():
    summary += str(sent)

print(summary)

Along these lines, tools such as environmental Web portals, blogs, wikis, and interactive
simulations of the environmental impact of an
activity could offer assistance.However, going
forward, the IT industry will need to deal with
all of the infrastructure requirements and the
environmental impact of IT and its use.Green IT Resources

The following are some helpful electronic resources:

> C/lO’s “Green IT” page (http://advice.cio.com/
taxonomy/term/27/0) discusses how to stay out of
the red with an environmental approach.cgi?m=newsletter) presents how businesses are
examining how they use their resources and exploring ways to save energy, materials, and money with
green il:

> GreenBiz (www.greenbiz.com) is the leading information resource on how to align environmental
responsibility with business success.
